# Session 4

## Another Linear Model: The Returns to Education

In [ ]:
# Load relevant libraries.

%pylab inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.api import abline_plot
import patsy
import seaborn as sns
sns.set(context='notebook', style='whitegrid', palette='deep', font='sans-serif', font_scale=1, rc=None)
import urllib2 as url

## Recap: Multivariate Regression Analysis

It is trivial to extend the single-feature linear model to a linear model that simultaneously incorporates multiple features.  The interpretation of the results of a statistical model that uses multiple features is the same the interpretation of the partial derivative from the calculus of many variables: the effect of a small change in a particular feature on a label (or outcome).  

A model with $K$ features, $x_{ik}$ and label $y_i$:

$y_i=\sum_{k=1}^Kx_{ik}\cdot\beta_k+\epsilon_i = x_i^\prime \beta + \epsilon_i$

The $K$ features $x_{ik}$ influence the label $y_i$ through the $K$-vector, $\beta$, which we estimate statistically.  A specific partial derivative interpretation.

${\displaystyle \frac{\partial E(y_i)}{\partial x_{ik}}=\beta_k}$

(For those interested in ancient history: Frisch–Waugh–Lovell theorem.)

Bottom line is simple: Fit the linear model with multiple features.  The basic approach to hypothesis testing remains unchanged. The challenge is the interpretation of the results, which we will discuss in detail.  

In [ ]:
# Another application: Why does someone go to school 
# (and should it be publicly subsidized)?
# Load Griliches data
# Paper is here: http://web.stanford.edu/~pista/griliches71.pdf
# Bio is here: https://en.wikipedia.org/wiki/Zvi_Griliches

griliches = pd.read_csv('griliches.csv')

In [ ]:
# DATA DICTIONARY
# Here: http://rpackages.ianhowson.com/cran/SciencesPo/man/griliches76.html
 
print griliches.info()

In [ ]:
# Summary Statistics

print griliches.describe()

In [ ]:
# As a dataframe, easy to plot.

plt.figure()
griliches['lw'].hist(bins = 50)
plt.title("Histogram of Log Wages", size = 20)

In [ ]:
plt.figure()
griliches['s'].hist(bins = 50)
plt.title("Histogram of Education", size = 20)

In [ ]:
# Start with bivariate linear model.

mod = smf.ols(formula='lw ~ s', data = griliches).fit()
print(mod.summary())

# Every additional year of education increases wages by about 10%.
# 95% Confidence Interval is [0.085, 0.108]

In [ ]:
# We don't need to stay in this limited world.
# Coding help: Generating indicator variables using Pandas.

print pd.get_dummies(griliches['s'] == 16).head(20)
griliches['undergrad'] = pd.get_dummies(griliches['s'] == 16)[1]
print
print griliches['undergrad'].head(20)

mod = smf.ols(formula='lw ~ rns + mrt + smsa + med + iq + age + tenure + s', data = griliches).fit()
print
print(mod.summary())

# Notice that taking into account "ability" through an IQ score cuts in half the measured return to education.

# The Linear Probability Model and The Logistic Classifier (or "Logit")

The problem above examined a continuous label or dependent variable.  That is, $y$ was quantitative: the sales prices of single-family houses in Staten Island.  Often, we wish to examine a discrete or categorical label.  What is a categorical label?  An example would be eye color: $\{$brown, green, blue$\}$.  

In economics, this is considered to be analysis of "limited dependent variables," while in computer science, it is classification.  This different terminology refers to the same thing.  We will begin with the linear probability model (with multiple attributes), which allows us to seemless transition to the Logistic Classifier (or "Logit").

This simple classifier is the ground upon which data scientists of all stripes converge.  It is a very important methodological technique that I want you to understand thoroughly.

## Latent Values

One can think of categorical variable as being driven by an underlying DGP, and we as observers, observe only outcomes.

Consider: $y_i^*=x_i^\prime\beta+\epsilon_i$

$y_i^*$ is a latent value for person i that is unobserved by us.  For example, you do not observe the value I place on this bottle of water.  You simple observe me drinking it.  In other words, you observe the outcome my choice set $\{$drink, not drink$\}$.  Another example would be observing someone taking a taxi.  You do not observe the value that someone places on the taxi ride.  You know that she has many alternative methods of transportation, all with varying values to her, but you observe the outcome of her choice.

$x_i$ are a vector of features (or attributes or predictors or independent variables).  We observe these.

$\beta$ is a vector that measures how features affect the latent index, which we will estimate statistically. 

$\epsilon_i$ retains its status as our ignorance.  

What do we observe?

${\displaystyle d_i = }$
$\left\{ \begin{array}{l l} 
{1} & \quad \text{if person i takes a cab, which happens when } y_i^*\ge0\\ 
{0} & \quad \text{if person i does not take a cab, which happens when } y_i^*\lt0 \\
\end{array} \right.$

The latent variable approach has become a very popular modeling tool in statistical learning.  

In [ ]:
# Read in some data on graduate school admissions.  These data are fictional, but useful.

data = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")
print data.info()
print data.describe()

In [ ]:
print data.corr()

In [ ]:
# School rank is a categorical feature, and we should capture this aspect using C(rank)
# C(rank) tells statsmodels to convert the categorical variable into indicator variables 
# and omit one of them
# The omitted categorical indicator is the reference category.
# Let's interpret the results from the linear probability model using the summary statistics 
# from the data.
# GRE: Graduate Record Examine: a one point increase in GRE increases probability of admission 
# by 0.0004 or 0.04%
# GPA: Grade Point Average: a one point increase in GPA increases probability of admission 
# by 0.156 or 15.6%.  
# rank: Going to a tier 2 school reduces probabilty of admission by 0.1624 or 16.24% 
# (relative to a tier 1 school).
# Note new line of code to generate predicted values: the labels that would be predicted using the model's attributes as is.

mod = smf.ols(formula='admit ~ gre + gpa + C(rank)', data = data).fit()
olshat = mod.predict(data)
print(mod.summary())

In [ ]:
# Let's make a prediction of admission for the "average" applicant (over all) coming from a tier 2 school.
# How might we generate a 95% confidence interval on this prediction?  You did it in Homework 2.  It's called bootstrapping.
# Remember the prediction for later.

print data.describe()
print 
print mod.params
print
print (mod.params['Intercept'] + mod.params['C(rank)[T.2]'] + mod.params['gre'] *587.7 + mod.params['gpa'] * 3.3899)

## The Logistic Classifier

Again, we start with a sample of size, $N$.  We are going to change the linear model to the following:

$Pr(d_i=1)=x_i^\prime\beta+\epsilon_i$

We will then impose a distributional assumption on $\epsilon_i$, namely that it is logistically distributed.

${\displaystyle Pr(d_i=1) = \frac{\exp(x_i^\prime\beta)}{1+\exp(x_i^\prime\beta)}}$ 

We see immediately that this is NOT a linear model (that is, a model that is linear in $\beta$).

${\displaystyle Pr(d_i=0) = 1 - Pr(d_i=1) = 1 - \frac{\exp(x_i^\prime\beta)}{1+\exp(x_i^\prime\beta)} = \frac{1}{1+\exp(x_i^\prime\beta)}}$

An "odds ratio":

${\displaystyle \frac{Pr(d_i=1)}{Pr(d_i=0)} = \frac{Pr(d_i=1)}{1 - Pr(d_i=1)} = \exp(x_i^\prime\beta)}$

This implies that the log-odds ratio (or "logit") is:

${\displaystyle \log\big(\frac{Pr(d_i=1)}{1 - Pr(d_i=1)}\big) = x_i^\prime\beta}$, which is linear in $\beta$.

To address the estimation of the parameters of interest, we need to construct a likelihood function that we will tell the computer to optimize.  Start by constructing the likelihood for observation $i$:

${\displaystyle l_i = Pr(d_i=1)^{d_i}\cdot Pr(d_i=0)^{(1-d_i)}=\frac{\exp(x_i^\prime\beta)}{1+\exp(x_i^\prime\beta)}^{d_1}\frac{1}{1+\exp(x_i^\prime\beta)}^{(1-d_i)}}$

If we make some assumptions we can write the likelihood function for the sample, $L$:

${\displaystyle L = \prod_{i=1}^N l_i = \prod_{i=1}^N \frac{\exp(x_i^\prime\beta)}{1+\exp(x_i^\prime\beta)}^{d_1}\frac{1}{1+\exp(x_i^\prime\beta)}^{(1-d_i)}}$

Goal is to tell the computer to maximize $L$ with respect to $\beta$ given the data we have. 

Once we have done that, we can make probabilistic predictions (or classifications).

In [ ]:
# Let's fit the logit and generate predicted values.

logit_mod = smf.logit('admit ~ gre + gpa + C(rank)', data = data).fit()
print
print(logit_mod.summary())
logithat = logit_mod.predict(data)

What are these logit coefficients?
Big lesson: Unlike the linear probability model, shown below, they cannot be directly interpreted.
Note, however, the similarities.  

1. The signs are identical.
2. The relative magnitudes are similar.
3. The z and the t are (relatively) similar.

In [ ]:
print(mod.summary())

In [ ]:
# Again we can make a prediction, the probability of admission or Pr(d=1), 
# using the logistic formula above.
# Average features at rank 2 school.

print data.describe()
print 
print logit_mod.params
print
e = np.exp(logit_mod.params['Intercept'] + logit_mod.params['C(rank)[T.2]'] + 
           logit_mod.params['gre'] *587.7 + logit_mod.params['gpa'] * 3.3899)
print(e / (1 + e))

In [ ]:
# We can also generate marginal effects, 
# which are the change in probability of admission given a "small" change in the features. 
# Note their similarilities to the regression estimates from the linear probability model.

marginal = logit_mod.get_margeff()
print(marginal.summary())
print
print mod.params

In [ ]:
# They hit it on average (as they should).

print data['admit'].mean(), olshat.mean(), logithat.mean()

In [ ]:
plt.figure(figsize=(12, 10))
plt.plot(olshat, logithat, 'bo')
plt.xlim(-.1, .8)
plt.ylim(-.1, .8)
plt.xlabel(r'$LPM$', fontsize = 16)
plt.ylabel(r'$Logit$', fontsize = 16)
plt.title(r'Comparison of Logit and LPM', fontsize = 20)
plt.axvline(0.0, color='black', ls='--', lw=2.0)
plt.axvline(0.3175, color='g', ls='--', lw=1.0)
plt.axhline(0.0, color='black', ls='--', lw=2.0)
plt.axhline(0.3175, color='g', ls='--', lw=1.0)

## Violations of Assumptions

To talk about violations, we need to state explicitly the assumptions that underlie our approach to date.

1. A linear model with $K$ features, $y_i = x_i^\prime \beta + \epsilon_i$ or $y=X\beta + \epsilon$.
2. $E(\epsilon) = 0$: errors are on average zero.
3. $Var(\epsilon) = \sigma^2 I_N$.  (Spherical errors.)
4. $Cov(X, \epsilon) = 0$
5. $\epsilon \sim N(0, \sigma^2 I_N)$.  This is a very strong assuption and wholely un-necessary.  People like to assert to because it implies that least squares and maximum likelihood are equivalent.  

If 1 through 4 above hold, then least squares is "BLUE": the Best Linear Unbiased Estimator.  You cannot do better.  If 5 is added to the mix, least squares is "MVUE": the Minimum Variance Unbiased Estimator.  (See Cramer Rao bound: https://en.wikipedia.org/wiki/Cram%C3%A9r%E2%80%93Rao_bound)

What matters here and why?

1. One reason we have developed a large number of arrows for our quiver of statistical learning.  One appeal of the linear model is interpretation of partial derivative effects.  If goal is prediction rather than having point estimates, other statistcal learning techniques better if more costly from computational point of view.  (Much less of an issue now, as "we never turn off the machine.")
2. Not really.  
3. Heteroscedasticity (or heteroskedasticity).  If our errors are non-spherical, we are not accurately measuring the standard errors that we need to do hypothesis testing.  Therefore, we may not be able to make valid statistical inferences.  We have developed methods to account for this (see https://en.wikipedia.org/wiki/Heteroscedasticity-consistent_standard_errors) that can be easily invoked using Statsmodels (Python) or GLS (R).   
4. Omitted Variable Bias.  For me, this is a very important issue.  In the homework, you estimated a toy model that suffered from omitted variable bias.  The presence of this bias would affect both hypothesis testing and prediction.  It would also impact other techniques developed in associated with a violation of 1.  
5. Not really.